# A/B Testing

## Background

Data scientists sometimes need to do experiments about user interface in web design and product marketing. A/B testing is a common approach to test the hypothesis. For example, whether a new interface B is better than the existing interface A.

**Control group** is a standard, existing, without-new-things group.

**Treatment group (Test group)** is a group with a new feature or a specific thing we want to test. 

A/B testing typically measures, for example

- Number of clicks
- Number of purchases
- Duration of seeing web apges

## Hypothesis test (Significance test)

Something that A/B testing usually establishes. When we see difference between A and B, the hypothesis test will tell us *whether it's due to random chance or true difference*. 

A/B testing typically uses **one-way** hypothesis test (**one-tail** hypothesis test). One-way hypothesis test treats one group as default and test whether the other group is better.

**Z-test** and **t-test** are hypothesis test. Both are the *same* in the following points.

- Decide if the **mean** from the sample is different from a certain value (or more than a value, or less than a value)
- Assume the samples follow a **normal distribution**.
- Use the same test statistic below. $s$ is the standard deviation. $n$ is the sample size

$$
\frac{\hat{\mu} - \mu_0}{\frac{s}{\sqrt{n}}}
$$

But they are *different* in the following.

Z-test uses **standard normal distribution**.

t-test uses **t-distribution**. It has **degrees of freedom**. As it gets large, t-distribution gets closer to standard normal distribution. When the sample size is small, t-test is more appropriate than Z-test.

In hypothesis test, 2 types of error can occur

### Type I error

Incorrectly conclude that an effect is real when it's actually just due to chance.

Reject the null hypothesis when the null hypothesis is true.

For example, the hypothesis test is about whether a drug has an effect on a patient. Null hypothesis is that the drug has no effect. The alternative hypothesis is that the drug has an effect. Type I error in this hypothesis test is to conclude that the drug has an effect (when the drug has no effect).

It's also called **false positive rate**.

In the hypothesis test, we typically don't wanna be fooled by the random chance, so we try to minimize this Type I error more than the following Type II error.

### Type II error

Incorrectly conclude that an effect is not real, so the effect is just due to chance, but it's actually real.

Fail to reject the null hypothesis when the null hypothesis is false.

For example, in the same testing a drug, Type II error is to conclude that the drug has no effect, when the drug actually has effect.

It's also called **false negative rate**.

### p-value

Given a null hypothesis model, the probability that results as extreme as the observed results might occur.

Given a chance model, p-value is the probability that a result is this extreme.

In random permutation test, the proportion of times that the test produces a difference equal to or greater than the observed difference.

The wrong interpretation of p-value is the probability that the result is due to chance.

## Random permutation test

Also called, **randomization test**, **resampling permutation procedure**, **bootstrap permutation test**.

**Permute** means to change the order of a set of values.

Different way from Z-test and t-test to test whether an observed test statistic is significant or not. I describe the methodology here, but I think looking at the below code is easier to understand.

1. Combine group A and B, shuffle and draw sample with the same size as the number of data in group A.
2. The remaining data is group B.
3. Compute a test statistic and save it
4. Repeat above steps many times to create a distribution
5. The distribution works as the reference null hypothesis distribution.
6. Compare the original observed test statistic with this distribution to compare p-value.

*Useful when samples don't follow a normal distribution*. So when we can't use Z-test or t-test, we can use random permutation test instead.

```python
import pandas as pd


df = pd.DataFrame('web_page_data.csv')
```

![Web page data](./web_page_data.png)

```python
mean_a = df.loc[df['Page'] == 'Page A', 'Time'].mean()
mean_b = df.loc[df['Page'] == 'Page B', 'Time'].mean()
observed_difference = mean_b - mean_a
print(f'Observed difference: {observed_difference}')


def permute_and_compute_difference(df, n_a, n_b):
    # Assumes dataframe has index from 0 to n - 1
    indices = [i for i in range(n_a + n_b)]
    index_b = set(random.sample(indices, n_b))
    index_a = set(indices) - index_b
    mean_a = df.loc[index_a, 'Time'].mean()
    mean_b = df.loc[index_b, 'Time'].mean()
    return mean_b - mean_a


n_a = len(df.loc[df['Page'] == 'Page A'])
n_b = len(df.loc[df['Page'] == 'Page B'])
num_sampling = 1000
permuted_differences = []

for _ in range(num_sampling):
    difference = permute_and_compute_difference(df, n_a, n_b)
    permuted_differences.append(difference)

p_value = np.mean(observed_difference < permuted_differences)
```

```python
import matplotlib.pyplot as plt


plt.hist(
    permuted_differences, 
    bins=30,
    label='Permuted differences'
)
plt.axvline(
    observed_difference,
    color='tab:orange',
    linestyle='--',
    label='Observed difference'
)
plt.legend()
plt.show()
```

![Random permutation test](./random_permutation_test.png)

## Multiple A/B tests

We can run multiple A/B tests, but as we run multiple testing, it increases the probability of making Type I error. This is called **alpha inflation**.

When $\alpha$ is the probability that falsely test is significant, $(1 - \alpha)$ is the probability that correctly test is not significant. When we run $n$ tests and all are correctly tested as non-significant, the probability is $(1 - \alpha)^n$. So the probability that at least one test will falsely test significant is $1 - (1 - \alpha)^n$. When $\alpha = 0.05$ and $n = 20$, 

$$
1 - (1 - 0.05)^{20}
$$
$$
= 1 - 0.95^{20}
$$
$$
= 1 - 0.36
$$
$$
= 0.64
$$

So the probability of making at least one type 1 error in multiple testing is high.

For multiple A/B testing, we can try **Bonferroni correction**, **F-test**, and **multi-arm bandit**. 

### Bonferroni correction (adjustment)

Divide significance level $\alpha$ by the number of multiple test $n$, so $\frac{\alpha}{n}$.

In Bonferroni correction, when we run $n$ tests with significance level $\alpha$, we use a significance level $\frac{\alpha}{n}$ instead. Even if all the hypotheses are false positive, the significance level is $\alpha$.

The disadvantage is to increase false negative. It's type II error. It's problematic if false positive is preferable to false negative. For example, in healthcare, false positive result of detecting disease gives concern and more tests might be required, but false negative means ignoring a potentially fetal disease.

### F-test

F-test can run hypothesis tests at once. But F-test only tests us whether there is a difference in mean between at least 2 groups, and it doesn't tell us how much differ, which groups, how many groups.

The disadvantage is the same as above Bonferroni correction.

### Multi-arm bandit algorithm

As we proceed making tests, suppose we have variant A, B, and C, if we find A is better, we start using A in production more often, but by random manner, sometimes we are still use B and C. Continuously doing this try and error.

**Epsilon-greedy algorithm for A/B test**

1. Set epsilon somewhere between 0 and 1, but to be a small number.
2. Get a random number between 0 and 1.
3. If the random number < epsilon, flip a coin
  1. If head, use A
  2. If tail, use B
4. If the random number $\ge$ epsilon, use an option which gives us the better result so far.

When epsilon is 1, it's a regular A/B testing, random allocation. When epsilon is 0, it's a greedy algorithm. Always choose the best variant so far, no more experiment.

## Compute sample size

We can compute sample size if we have **effect size**, **power**, and **significance level**.

The **effect size** is the amount of change we hope to have with a new feature. For example, with a new feature, if we want the click rate to increase by 10%, the 10% is the effect size.

The **power** is the probability of detecting the effect size in A/B testing.

The **significance level** is the probability of incorrectly saying we have the effect when the effect actually doesn't exist.

## Experimental design

### Multiple features

Button color and button location

### Interpret A/B testing result

new UI effect

### Apply A/B testing to price

### Randomization

Random bucketing

### Non-normal distribution

Mann-whitney, etc.

## Terminology

- Churn rate
  - The rate at which a customer stops doing business with a company, or stops subscribing to a service.
- Researcher bias
  - Selecting a test statistic after an experiment, but the test statistic needs to be established beforehand. 

## Reference

- O'REILLY Practical Statistics for Data Scientist, 50+ Essential Concepts Using R and Python
- [How to A/B Test Your Pricing (And Why It Might Be a Bad Idea)](https://blog.hubspot.com/marketing/price-testing)
- [Common Mistakes During A/B Testing](https://towardsdatascience.com/common-mistakes-during-a-b-testing-bdb9eefdc7f0)
- [Type I & Type II Errors | Differences, Examples, Visualizations](https://www.scribbr.com/statistics/type-i-and-type-ii-errors/)